In [1]:
import pandas as pd
import numpy as np

In [10]:
bureau = pd.read_csv('credit_risk/bureau.csv')

In [11]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [12]:
bureau_cat_feats = [
    'CREDIT_CURRENCY', 
    'CREDIT_TYPE', 
    'CREDIT_ACTIVE'
]
cat_feats_df = bureau.loc[:,['SK_ID_CURR'] + bureau_cat_feats]

In [14]:
cat_feats_df.head()

,SK_ID_CURR,CREDIT_CURRENCY,CREDIT_TYPE,CREDIT_ACTIVE
0,215354,currency 1,Consumer credit,Closed
1,215354,currency 1,Credit card,Active
2,215354,currency 1,Consumer credit,Active
3,215354,currency 1,Credit card,Active
4,215354,currency 1,Consumer credit,Active


In [15]:
cat_feats_df[bureau_cat_feats] = cat_feats_df[bureau_cat_feats].apply(lambda x: x.astype('category'))

In [16]:
cat_feats_df.head()

,SK_ID_CURR,CREDIT_CURRENCY,CREDIT_TYPE,CREDIT_ACTIVE
0,215354,currency 1,Consumer credit,Closed
1,215354,currency 1,Credit card,Active
2,215354,currency 1,Consumer credit,Active
3,215354,currency 1,Credit card,Active
4,215354,currency 1,Consumer credit,Active


In [18]:
cat_aggregations = {}
for cat in bureau_cat_feats: 
    # Get numerical code of each categorical feature.
    cat_feats_df[cat] = cat_feats_df.loc[:,cat].cat.codes
    # Specify type of aggregation for cat features.
    cat_aggregations[cat] = ['mean']

In [19]:
cat_aggregations

{'CREDIT_CURRENCY': ['mean'],
 'CREDIT_TYPE': ['mean'],
 'CREDIT_ACTIVE': ['mean']}

In [30]:
latest_cat_feats_df = (
    bureau [
        ['SK_ID_CURR', 'DAYS_CREDIT'] + bureau_cat_feats
    ]
    .loc [
        bureau [
            ['SK_ID_CURR', 'DAYS_CREDIT'] + bureau_cat_feats
        ]
        .sort_values(['SK_ID_CURR','DAYS_CREDIT']).drop_duplicates('SK_ID_CURR',keep='last').index
    ]
)
    

In [31]:
from lightgbm import LGBClassifier

ModuleNotFoundError: No module named 'lightgbm'

In [1]:
import lightgbm